In [ ]:
!pip install transformers sentence-transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━

In [ ]:
import string
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# TEST
test_dataset = load_dataset("stsb_multi_mt", name="en", split="test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
def vectorize(text1, text2):
  vectorizer = TfidfVectorizer()
  tfidf_matrix = vectorizer.fit_transform([text1, text2])
  return tfidf_matrix

In [ ]:
actual_scores = []
predicted_scores = []

for example in test_dataset:
  tfidf_matrix = vectorize(example['sentence1'].lower(), example['sentence2'].lower())
  actual_similarity = example['similarity_score'] / 6.0  # Normalize actual similarity by 6

  similarity_score = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]

  actual_scores.append(actual_similarity)
  predicted_scores.append(similarity_score)

In [ ]:
# Create a dataframe
df = pd.DataFrame({'Actual': actual_scores, 'Predicted': predicted_scores})
df

,Actual,Predicted
0,0.50,0.669419
1,0.72,0.550780
2,1.00,0.615219
3,0.84,0.510149
4,0.30,0.602975
...,...,...
1374,0.00,0.084580
1375,0.20,0.145874
1376,0.20,0.135073
1377,0.00,0.119147


In [ ]:
# Define threshold for positive prediction
threshold = 0.5

# Calculate true positives, false positives, and false negatives
true_positives = ((df['Actual'] >= threshold) & (df['Predicted'] >= threshold)).sum()
false_positives = ((df['Actual'] < threshold) & (df['Predicted'] >= threshold)).sum()
false_negatives = ((df['Actual'] >= threshold) & (df['Predicted'] < threshold)).sum()

# Calculate precision and recall
precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1_score:.2f}")

Precision: 0.80
Recall: 0.50
F1 Score: 0.62
